# AWS Marketplace Product Usage Demonstration - Multivariate ForecastGPT Model Package

## ForecastGPT

This sample notebook demonstrates the usage of the ForecastGPT.

This solution is multivariate forecasting model to forecast data from a sample input data.

This sample notebook shows you how to deploy Multivariate forecast model using Amazon SageMaker.

**Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Car Mileage Predictor. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](# Subscribe-to-the-model-package)
2. [Set up the environment](# Setup the environment)
3. [Create the session](# Create the session)
4. [Create Model](# Create model)
5. [Create an endpoint and perform real-time inference](#-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
6. [Perform batch inference](# Perform-batch-inference)
    1. [Configure the input S3 bucket folder](# A. -Configure the input S3 bucket folder)
    2. [Deploy the model](# B. -Deploy the model)
    3. [Download the file from output S3 bucket folder](# C. -Download the file from output S3 bucket folder)
    4. [Visualize data](# D. -Visualize data)
7. [Clean-up](# Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Car Mileage Predictor. 
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## 2. Set up the environment

In [2]:
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from sagemaker.serializers import CSVSerializer
import boto3
import json

role = get_execution_role()

# S3 prefixes
common_prefix = "<your s3 bucket name>"
batch_inference_input_prefix = "forecastgpt/input"
batch_inference_output_prefix = "forecastgpt/output/"

## 3. Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our Amazon SageMaker operations.

In [3]:
sagemaker_session = sage.Session()

## 4. Create Model

Now we use the above Model Package to create a model

In [21]:
model_package_arn = '<your model package arn>'

In [8]:
model_name = 'forecastgpt'

content_type = 'application/json'

real_time_inference_instance_type = 'ml.m4.4xlarge' 

batch_transform_inference_instance_type = 'ml.m4.4xlarge'

In [9]:
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session, content_type)

# Create a deployable model from the model package
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls = predict_wrapper)

## 5. Create-an-endpoint-and-perform-real-time-inference 

#### A. Create an endpoint

In [10]:
#Deploy the model
predictor = model.deploy(1,real_time_inference_instance_type,endpoint_name = model_name)

-------!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [11]:
file_name = 'input/ett.json' 

In [12]:
with open(file_name, 'r') as f:
    data = json.load(f)
#data=json.
data

{'date': {'0': '2016-07-01 00:00:00',
  '1': '2016-07-01 01:00:00',
  '2': '2016-07-01 02:00:00',
  '3': '2016-07-01 03:00:00',
  '4': '2016-07-01 04:00:00',
  '5': '2016-07-01 05:00:00',
  '6': '2016-07-01 06:00:00',
  '7': '2016-07-01 07:00:00',
  '8': '2016-07-01 08:00:00',
  '9': '2016-07-01 09:00:00',
  '10': '2016-07-01 10:00:00',
  '11': '2016-07-01 11:00:00',
  '12': '2016-07-01 12:00:00',
  '13': '2016-07-01 13:00:00',
  '14': '2016-07-01 14:00:00'},
 'HUFL': {'0': 5.8270001411,
  '1': 5.6929998398,
  '2': 5.1570000648,
  '3': 5.0900001526,
  '4': 5.3579998016,
  '5': 5.6259999275,
  '6': 7.1669998169,
  '7': 7.4349999428,
  '8': 5.5590000153,
  '9': 4.5549998283,
  '10': 4.9569997787,
  '11': 5.7600002289,
  '12': 4.6890001297,
  '13': 4.6890001297,
  '14': 5.0900001526},
 'HULL': {'0': 2.0090000629,
  '1': 2.0759999752,
  '2': 1.7410000563,
  '3': 1.9420000315,
  '4': 1.9420000315,
  '5': 2.1429998875,
  '6': 2.9470000267,
  '7': 3.2820000648,
  '8': 3.013999939,
  '9': 2.54

#### C. Perform real-time inference

In [13]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [14]:
import csv
import json

data = []

with open('output.json', 'r') as f:
    data = json.load(f)
for i,v in data.items():
    if i=='result':
        result=v
result   # forecasted output

[[5.131582736968994,
  2.957359790802002,
  2.3567450046539307,
  1.3344697952270508,
  2.841761589050293,
  1.5155383348464966,
  19.937015533447266],
 [5.1317620277404785,
  2.9284403324127197,
  2.3371293544769287,
  1.2956286668777466,
  2.82720947265625,
  1.4990074634552002,
  19.59956169128418],
 [5.163863658905029,
  2.878535032272339,
  2.3105084896087646,
  1.2562291622161865,
  2.8217320442199707,
  1.4863511323928833,
  19.501346588134766],
 [5.056051254272461,
  2.8135616779327393,
  2.2357633113861084,
  1.1918236017227173,
  2.7262022495269775,
  1.4806846380233765,
  18.788660049438477],
 [5.114974498748779,
  2.787313938140869,
  2.2219409942626953,
  1.128020167350769,
  2.8188819885253906,
  1.4553855657577515,
  19.4351863861084],
 [5.121236801147461,
  2.726931095123291,
  2.1498310565948486,
  1.096701741218567,
  2.8770666122436523,
  1.473987340927124,
  19.699953079223633],
 [5.074916362762451,
  2.624584436416626,
  2.071563482284546,
  1.0045452117919922,
  2

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [23]:
predictor.delete_endpoint(delete_endpoint_config = True)

INFO:sagemaker:Deleting endpoint configuration with name: forecastgpt
INFO:sagemaker:Deleting endpoint with name: forecastgpt


## 6. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

### A. Configure the input S3 bucket folder

In [15]:
transform_input_folder = 'input'
transform_input = sagemaker_session.upload_data(transform_input_folder,common_prefix,batch_inference_input_prefix)

### B. Deploy the model

In [16]:
transformer = model.transformer(1,batch_transform_inference_instance_type,output_path="s3://"+common_prefix+"/"+batch_inference_output_prefix+"/")
transformer.transform(transform_input,content_type=content_type)
transformer.wait()

INFO:sagemaker:Creating transform job with name: ForecastGPT-2024-09-27-13-57-18-464


................................Starting the inference server with 16 workers.
[2024-09-27 14:02:40 +0000] [10] [INFO] Starting gunicorn 23.0.0
[2024-09-27 14:02:40 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2024-09-27 14:02:40 +0000] [10] [INFO] Using worker: sync
[2024-09-27 14:02:40 +0000] [13] [INFO] Booting worker with pid: 13
[2024-09-27 14:02:40 +0000] [14] [INFO] Booting worker with pid: 14
[2024-09-27 14:02:40 +0000] [30] [INFO] Booting worker with pid: 30
[2024-09-27 14:02:40 +0000] [46] [INFO] Booting worker with pid: 46
[2024-09-27 14:02:40 +0000] [47] [INFO] Booting worker with pid: 47
[2024-09-27 14:02:40 +0000] [63] [INFO] Booting worker with pid: 63
[2024-09-27 14:02:40 +0000] [79] [INFO] Booting worker with pid: 79
[2024-09-27 14:02:40 +0000] [80] [INFO] Booting worker with pid: 80
[2024-09-27 14:02:40 +0000] [96] [INFO] Booting worker with pid: 96
[2024-09-27 14:02:40 +0000] [127] [INFO] Booting worker with pid: 127
[2024-09-27 14:02:40 +0000] [143

### C. Download the file from output S3 bucket folder

In [19]:
import os
s3_conn = boto3.client("s3")
with open('output3.json','wb') as f:
    s3_conn.download_fileobj(common_prefix,batch_inference_output_prefix+'ett.json.out',f) # ett.json.out = this file will be the same input file name 
    print("Output file loaded from bucket")

Output file loaded from bucket


### D. Visualize data

In [20]:
import csv
import json

data = []

with open('output3.json', 'r') as f:
    data = json.load(f)
for i,v in data.items():
    if i=='result':
        result=v
result  

[[5.131582736968994,
  2.957359790802002,
  2.3567450046539307,
  1.3344697952270508,
  2.841761589050293,
  1.5155383348464966,
  19.937015533447266],
 [5.1317620277404785,
  2.9284403324127197,
  2.3371293544769287,
  1.2956286668777466,
  2.82720947265625,
  1.4990074634552002,
  19.59956169128418],
 [5.163863658905029,
  2.878535032272339,
  2.3105084896087646,
  1.2562291622161865,
  2.8217320442199707,
  1.4863511323928833,
  19.501346588134766],
 [5.056051254272461,
  2.8135616779327393,
  2.2357633113861084,
  1.1918236017227173,
  2.7262022495269775,
  1.4806846380233765,
  18.788660049438477],
 [5.114974498748779,
  2.787313938140869,
  2.2219409942626953,
  1.128020167350769,
  2.8188819885253906,
  1.4553855657577515,
  19.4351863861084],
 [5.121236801147461,
  2.726931095123291,
  2.1498310565948486,
  1.096701741218567,
  2.8770666122436523,
  1.473987340927124,
  19.699953079223633],
 [5.074916362762451,
  2.624584436416626,
  2.071563482284546,
  1.0045452117919922,
  2

### 4. Clean-up

### A. Delete the model

In [22]:
model.delete_model()

INFO:sagemaker:Deleting model with name: ForecastGPT-2024-09-27-13-57-17-733


#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.